<a href="https://colab.research.google.com/github/SeongGwangJu/SeongGwangJu/blob/main/%EB%A7%8C%EB%93%A4%EB%A9%B4%EC%84%9C_%EB%B0%B0%EC%9A%B0%EB%8A%94_AI_%EC%84%9C%EB%B9%84%EC%8A%A4_%EC%8B%A4%EC%8A%B52.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 패키지 설치
Langchain, Google-genai 패키지를 설치합니다.

In [ ]:
!pip install -q chromadb google-generativeai langchain-google-genai langchain_community pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 12.2 MB/s eta 0:

### API 키 등록
Gemini API를 사용하기 위해서 Google_API_Key를 불러옵니다.
Colab 왼쪽 패널의 `보안 비밀`, `Gemini API 키`, `Google AI Studio에서 키 가져오기`를 선택해 키를 불러옵니다.

In [ ]:
from google.colab import userdata
import os

# Secrets에 저장된 값을 가져옵니다.
my_api_key = userdata.get('GOOGLE_API_KEY')

# os.getenv()를 사용하는 라이브러리와의 호환을 위해 os.environ에 설정해줍니다.
os.environ['GOOGLE_API_KEY'] = my_api_key

# 이제 os.getenv()로 값을 읽을 수 있습니다.
retrieved_key = os.getenv('GOOGLE_API_KEY')
print("키를 성공적으로 불러왔습니다.")

키를 성공적으로 불러왔습니다.


# RAG : Retrieval-Augmented Generation

RAG는 모델 내부의 지식에만 의존하지 않고 외부의 최신 또는
전문적인 정보를 실시간으로 검색하여 그 결과를 바탕으로 답변을 생성하는 기법입니다.

PDF 파일에서 유사도 검색을 통해 RAG를 적용해보겠습니다.

### Colab에 파일 업로드하기

In [ ]:
from google.colab import files

uploaded = files.upload()

# 업로드된 파일 이름 가져오기
file_name = next(iter(uploaded))

# 파일 내용(바이트)을 변수에 저장
file_content = uploaded[file_name]

print(f'파일 이름: {file_name}')
print(f'파일 크기: {len(file_content)} 바이트')
print(f'파일 내용(일부): {file_content[:50]}...')

Saving 18859_2.pdf to 18859_2.pdf
파일 이름: 18859_2.pdf
파일 크기: 3191319 바이트
파일 내용(일부): b'%PDF-1.6\r%\xe2\xe3\xcf\xd3\r\n2667 0 obj\r<</Linearized 1/L 31913'...


In [ ]:
import os
from google.colab import files, userdata
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

**PyPDFLoader**는 PDF 문서를 로드하는 역할을 합니다. 이 로더를 사용하면 PDF 파일의 텍스트와 메타데이터를 LangChain이 처리할 수 있는 Document 객체로 변환할 수 있습니다.

**RecursiveCharacterTextSplitter**는 긴 문서를 작은 텍스트 덩어리(chunk)로 나누는 역할을 합니다.  LLM은 한 번에 처리할 수 있는 텍스트 양에 한계가 있어, 이 스플리터를 사용해 문서를 효과적으로 분할해야 합니다.

**Chroma**는 텍스트 임베딩을 저장하고 검색하는 벡터 데이터베이스입니다. PyPDFLoader로 로드하고 RecursiveCharacterTextSplitter로 분할된 텍스트 청크들을 임베딩으로 변환한 후, 이 벡터 데이터베이스에 저장하여 나중에 사용자의 질문과 관련된 문서를 빠르게 찾아낼 수 있습니다.

In [ ]:
pdf_file_path = list(uploaded.keys())[0]
print(f"'{pdf_file_path}' 파일이 업로드되었습니다.")

'18859_2.pdf' 파일이 업로드되었습니다.


# Vector DB 만들기
RAG를 사용하기 전, Vector DB를 먼저 구축해야 합니다.

Vector DB는 준비된 파일로부터 텍스트를 로드, 분할, 임베딩 과정을 거쳐서 생성됩니다.

`Text load` -> `Text split` -> `Text embedding`

### PDF 파일 로드:
`PyPDFLoader`로 PDF 파일을 준비하고, LangChain이 사용할 수 있는 형식으로 변환합니다.

### 텍스트 분할:
`RecursiveCharacterTextSplitter`를 사용해 긴 문서를 작은 텍스트 조각들로 나눕니다.

`chunk_size=1000`은 각 조각의 크기를 약 1,000자로 정합니다. LLM은 한 번에 처리할 수 있는 텍스트 양이 제한되어 있어 이렇게 나누는 것이 필수적입니다.

`chunk_overlap=200`은 앞뒤 조각이 200자 정도 겹치게 하여, 문맥이 끊기지 않도록 도와줍니다.

### 결과 확인:
`text_splitter.split_documents(documents)`는 문서를 나누는 과정입니다.

In [ ]:
# PDF 로드 및 텍스트 분할을 시작합니다.
print("PDF 로드 및 텍스트 분할")

# PyPDFLoader를 사용하여 지정된 경로의 PDF 파일을 로드할 준비를 합니다.
loader = PyPDFLoader(pdf_file_path)

# PDF 파일의 모든 페이지를 불러와서 'documents' 변수에 저장합니다.
# 각 페이지는 LangChain의 Document 객체가 됩니다.
documents = loader.load()

# 텍스트를 나눌 기준을 정의하는 RecursiveCharacterTextSplitter를 생성합니다.
# chunk_size는 텍스트 덩어리의 최대 크기를, chunk_overlap은 덩어리 간의 겹치는 부분을 지정합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200) # ⭐ 파라미터를 바꾸어서 테스트 해보세요! 결과는 바로 아래 코드 (print(texts[11])) 에서 청크 사이즈를 확인할 수 있습니다.

# 로드된 문서를 위에서 정의한 기준으로 작은 텍스트 덩어리(청크)로 분할합니다.
texts = text_splitter.split_documents(documents)

# 분할된 텍스트 덩어리의 총 개수를 출력하여 확인합니다.
print(f"총 {len(texts)}개의 텍스트 청크로 분할되었습니다.")

PDF 로드 및 텍스트 분할
총 17개의 텍스트 청크로 분할되었습니다.


아래와 같이, 11번째 청크를 확인할 수 있습니다.

In [ ]:
print(type(texts[11]))
print(texts[11])

<class 'langchain_core.documents.base.Document'>
page_content='Ⅱ. 2025~26년 국내경제 전망
11
1. 대외여건에 대한 주요 전제
2025~26년 세계경제는 미국 관세인상 등으로 낮은 성장세에 머무를 것으로 전제
최근 IMF는 금년과 내년의 세계경제 성장률을 각각 3.0%, 3.1%로 전망하였으며, 이는 
코로나19 위기 이전(2011~19년)의 평균 성장률(3.5%)보다 낮은 수준임.
최근 WSTS는 2025년 메모리반도체 매출액 증가율을 기존(2025년 상반기 KDI 경제
전망) 전제(13.4%)보다 높은 17.1%로 전망
미국 관세율은 현재 정해진 수준이 향후에도 유지될 것으로 전제하였으며, 이에 따라 
우리 상품에 대한 평균 관세율은 기존 전제(14.1%)와 유사한 14.5%로 전제
╺ 최근 발표된 반도체 관세는 세부 사항이 불명확하여, 이번 전망에는 반영되지 않음.
원유 도입단가(두바이유 기준)는 수요 부진에 주로 기인하여 2024년의 배럴당 80달러
에서 2025년 71달러, 2026년 67달러로 하락할 것으로 전제
다만, 지정학적 위험　확대에 따른 유가 상승을 반영하여 기존 전제(69달러, 66달러)
보다는 소폭 상향 조정
실질실효환율로 평가한 원화가치는 최근 수준에서 큰 변동이 없을 것으로 전제
글로벌 메모리반도체 매출액 증가율 전제 국제유가 전제
  주: 기존 전제는 2025년 상반기(5월) KDI 경제전망 전제이며, 국제유가는 두바이유 기준임.
자료: WSTS; 한국석유공사; KDI 자체 전망.' metadata={'producer': 'Adobe Acrobat (64-bit) 25.1.20577', 'creator': 'Adobe Acrobat (64-bit) 25.1.20577', 'creationdate': '2025-08-12T17:23:03+09:00', 'author': 'master', 'moddate': '2025-08-12T17:32:06+09:00', 'source': 

이제, `GoogleGenerativeAIEmbeddings`를 활용해 임베딩을 생성합니다.

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# 임베딩 및 벡터 데이터베이스 생성
print("텍스트 임베딩 및 벡터 데이터베이스 생성 중...")

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vectordb = Chroma.from_documents(texts, embeddings)

print("임베딩 및 벡터 데이터베이스 생성되었습니다")

텍스트 임베딩 및 벡터 데이터베이스 생성 중...


# Retriver(검색기) 설정
--- **기본 유사도 검색(Similarity Search) 설정** ---

이 방식은 가장 직관적인 검색 방법으로, 사용자의 질문과 의미적으로
'가장 가까운(가장 유사한)' 문서 조각을 순서대로 찾아옵니다.

* search_type="similarity": 검색 유형을 유사도(기본값)로 설정합니다.
* search_kwargs={'k': 4}:
   - k=4: 질문과 가장 유사한 순서대로 문서 조각 4개를 찾아 LLM에게 전달합니다.

```python
retriever = vectordb.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)
```

참고: as_retriever()에 아무 옵션을 주지 않으면 이 설정과 동일하게 작동합니다.
```python
retriever = vectordb.as_retriever()
```
--- **MMR(Maximal Marginal Relevance) 검색기 설정** ---

단순 유사도 검색의 단점을 보완하기 위해 MMR 방식을 사용합니다.
MMR은 질문과의 '유사도'와 이미 선택된 문서들과의 '다양성'을 함께 고려하여,
목차나 거의 동일한 내용의 문서 조각이 중복으로 검색 결과에 포함되는 것을 방지합니다.

* search_type="mmr": 검색 알고리즘을 MMR로 지정합니다.
* search_kwargs={'k': 5, 'fetch_k': 20}:
   - fetch_k=20: 먼저 질문과 유사도가 높은 문서 후보 20개를 가져옵니다.
   - k=5: 가져온 후보 20개 중에서, 유사도와 다양성을 함께 평가하여 최종 5개를 선택해 LLM에게 전달합니다.

In [ ]:
retriever = vectordb.as_retriever(
    search_type="mmr",                      # ⭐ 알고리즘을 바꾸어서 테스트 해보세요!
    search_kwargs={'k': 5, 'fetch_k': 20}   # ⭐ 파라미터를 바꾸어서 테스트 해보세요!
)

Retriever(검색기)에 사용자 질의를 던져 유사한 문서를 확인할 수 있습니다.

이때, 유사한 문서를 가져오는 규칙은 정의한 알고리즘(`search_type`)에 따라 달라집니다.

In [ ]:
question = "2025~26년 국내경제 전망에 대해 알려주세요" # ⭐ 질문을 바꾸어서 테스트 해보세요!
retrieved_docs = retriever.invoke(question)

print("--- 검색된 문서 조각 (Retrieved Chunks) ---")   # 검색된 문서 조각을 확인할 수 있습니다.
for i, doc in enumerate(retrieved_docs):               # 문서 조각은 청크로 나뉘어서 출력됩니다. 이 결과가 프롬프트에 들어가게 됩니다.
    print(f"--- CHUNK {i+1} (Page: {doc.metadata.get('page', 'N/A')}) ---")
    print(doc.page_content)
    print("-" * 50)

--- 검색된 문서 조각 (Retrieved Chunks) ---
--- CHUNK 1 (Page: 13) ---
Ⅱ
2025~26년 국내경제 전망
1. 대외여건에 대한 주요 전제
2. 2025~26년 국내경제 전망
3. 전망의 위험요인
--------------------------------------------------
--- CHUNK 2 (Page: 15) ---
Ⅱ. 2025~26년 국내경제 전망
11
1. 대외여건에 대한 주요 전제
2025~26년 세계경제는 미국 관세인상 등으로 낮은 성장세에 머무를 것으로 전제
최근 IMF는 금년과 내년의 세계경제 성장률을 각각 3.0%, 3.1%로 전망하였으며, 이는 
코로나19 위기 이전(2011~19년)의 평균 성장률(3.5%)보다 낮은 수준임.
--------------------------------------------------
--- CHUNK 3 (Page: 19) ---
Ⅱ. 2025~26년 국내경제 전망
15
2025~26년 경제전망
(전년동기대비, %, %p, 억달러, 만명)
2024p 2025p 2026 2025 
수정폭2)
2026 
수정폭2)
연간p 상반기p 하반기 연간 연간
국내총생산 2.0 0.2 1.3 0.8 1.6 0.0 0.0 
총소비 1.4 1.2 2.0 1.6 1.7 0.2 0.0 
 민간소비 1.1 0.7 1.8 1.3 1.5 0.2 -0.1
총고정투자 -0.8 -4.0 -1.4 -2.7 2.2 -1.8 0.2 
 설비투자 1.7 4.8 -1.0 1.8 1.6 0.1 0.0 
 건설투자 -3.3 -12.4 -3.8 -8.1 2.6 -3.9 0.2 
 지식재산생산물투자 1.2 1.3 2.9 2.2 2.1 0.2 0.2 
총수출(물량) 6.8 2.8 1.4 2.1 0.6 1.8 -0.2
 상품수출(물량) 6.4 1.6 0.9 1.2 0.2 1.6 -0.3
총수입(물량) 2.5 3.1 0.9 2.0 1.2 1.2 -0.2
 상품수입(물량) 1.3 1.8 0.3 1.0 0.7 

이제, 프롬프트 템플릿을 정의하고 chain을 구축합니다.

In [ ]:
# 프롬프트 템플릿 정의
template = """
당신은 주어진 문서를 바탕으로 사용자의 질문에 답하는 유용한 AI 어시스턴트입니다.
검색된 참고문서를 먼저 알려주고,
참고 문서를 참고하여 사용자 질문에 답변해주세요.

참고 문서:
{context}

사용자 질문:
{query}

답변은 오직 제공된 참고 문서에 기반해야 합니다.
"""
prompt = ChatPromptTemplate.from_template(template)

# LLM 모델 정의
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0)

# 체인 구축: (검색기 | 프롬프트) | LLM
# RunnablePassthrough.assign()은 검색 결과를 'context' 변수로 전달합니다.
rag_chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
)

사용자의 질문을 정의하고, chain을 invoke 합니다.

In [ ]:

question = "2025~26년 국내경제 전망에 대해 알려주세요"
response = rag_chain.invoke(question)

print(f"사용자 질문: {question}")
print("---")
print(f"답변: \n{response.content}")
print("--------------------")
